最終考題測試，感覺效果不是很理想，最後仍然輸給 Google 爬蟲大法，覺得是原本的訓練集就很爛導致

In [1]:
import json
from ckip_transformers.nlp import CkipWordSegmenter
import re


In [2]:
# config

question = './questions_final_test.json'
reg = "[\s\-，.。\:：！!；;\?？（）\(\)\"\'《》〈〉．～—─\=「」『』、”“·／\#\[\]\$,－]"
db = './convert-to-db/inverted-index.db'

In [3]:
ws_driver = CkipWordSegmenter(level=1, device=0)


In [4]:
with open(question, encoding='utf8') as j:
    qs = json.load(j)


In [5]:
def wsAndPos(text): # text is string list
    for i, _ in enumerate(text):
        text[i] = re.sub(reg, " ", text[i])
    ws = ws_driver(text, show_progress=False)
    return ws


In [6]:
import sqlite3

conn = sqlite3.connect(db)
cursor = conn.cursor()

def findAidList(word):
    sql = "SELECT aids from mapping WHERE word = ?"
    res = cursor.execute(sql, (word,)).fetchone()
    if res is None:
        return []
    else:
        return res[0].split()

def getFrequency(word):
    sql = "SELECT num from mapping WHERE word = ?"
    res = cursor.execute(sql, (word,)).fetchone()
    return 0 if res is None else int(res[0])


In [7]:
def findAidListByWordList(wordList):
    ret = []
    for word in wordList:
        ret.append(findAidList(word))
    return ret


In [8]:
import math

# 自然指數最高可以到 710 次方
threshold = 700

def evaluate(questionAids, questionWords, answerAids):
    score = 0
    for a1, w1 in zip(questionAids, questionWords):
        # 當 frequency 過大時 exponential 也算不出來因此在此把他欄截
        # 透過此方式減少 intersection 計算量

        f = getFrequency(w1)
        if f > threshold:
            continue
        set1 = set(a1)
        for a2 in answerAids:
            set2 = set(a2)
            intersection = len(set1.intersection(set2))
            score += intersection / math.exp(f)            
    return score


In [9]:
myAnswer = []
for idx, q in enumerate(qs):
    # 將題目、選項都先進行斷詞
    wordList2D = wsAndPos(list(q.values()))
    
    scoreList = {}
    scoreSum = 1e-140
    maxK = 'C' # 預設答案猜 C
    maxV = 0
    for k, words in zip(q.keys(), wordList2D):
        if k == 'Question':
            questionAid = findAidListByWordList(words)
            questionWords = words
        else:
            answerAid = findAidListByWordList(words)
            score = evaluate(questionAid, questionWords, answerAid)
            scoreList[k] = score
            scoreSum += score
            if score > maxV:
                maxK = k
                maxV = score
    myAnswer.append(maxK)
    # normalize
    for k, v in scoreList.items():
        scoreList[k] = '%g%%' % (v/scoreSum*100,)


In [10]:
json.dumps(myAnswer)


'["A", "B", "B", "C", "A", "C", "A", "B", "B", "A", "B", "B", "B", "A", "B", "C", "B", "C", "C", "C", "C", "C", "C", "A", "C", "C", "C", "B", "A", "B", "A", "C", "A", "B", "A", "A", "C", "C", "C", "B", "C", "B", "A", "C", "A", "C", "C", "A", "C", "C", "B", "B", "C", "A", "C", "C", "C", "C", "A", "C", "B", "C", "C", "B", "A", "A", "C", "A", "B", "A", "A", "A", "C", "C", "C", "C", "B", "C", "C", "A", "A", "C", "C", "B", "B", "C", "A", "C", "C", "B", "B", "A", "C", "B", "C", "C", "C", "C", "B", "B", "A", "A", "B", "B", "A", "C", "B", "A", "C", "B", "C", "C", "A", "A", "C", "C", "C", "B", "A", "B", "C", "B", "B", "B", "B", "C", "C", "C", "B", "B", "A", "C", "B", "A", "A", "B", "C", "B", "C", "B", "C", "A", "C", "C", "A", "B", "C", "A", "B", "C", "B", "A", "A", "A", "C", "C", "A", "C", "B", "C", "B", "C", "C", "B", "A", "C", "C", "C", "A", "A", "A", "C", "C", "A", "B", "B", "C", "C", "C", "A", "C", "C", "C", "A", "A", "C", "B", "C", "B", "A", "C", "C", "A", "A", "B", "B", "C", "C", "A", "B"

In [11]:
same = 0
answer = ["A", "C", "B", "B", "C", "B", "A", "B", "B", "A", 
          "C", "B", "B", "A", "C", "A", "B", "C", "C", "B", 
          "A", "C", "C", "A", "B", "C", "A", "B", "A", "A", 
          "B", "A", "B", "B", "C", "A", "B", "C", "B", "A", 
          "A", "B", "B", "C", "A", "B", "B", "A", "C", "C", 
          "C", "B", "C", "A", "B", "C", "C", "A", "A", "C", 
          "A", "C", "B", "A", "A", "C", "C", "A", "B", "A", 
          "C", "A", "B", "A", "B", "C", "B", "B", "C", "B", 
          "C", "C", "B", "B", "C", "A", "C", "B", "A", "B", 
          "B", "A", "C", "B", "C", "C", "C", "A", "B", "B",
          "A", "A", "B", "C", "A", "C", "B", "C", "A", "C", 
          "C", "C", "A", "B", "C", "A", "C", "B", "A", "B", 
          "C", "B", "B", "B", "B", "A", "B", "C", "C", "B", 
          "A", "C", "C", "B", "A", "C", "A", "A", "C", "B", 
          "C", "A", "C", "A", "B", "A", "C", "A", "B", "C", 
          "C", "A", "B", "A", "C", "C", "A", "C", "B", "C", 
          "B", "C", "A", "C", "C", "C", "B", "B", "A", "B", 
          "A", "A", "A", "A", "A", "B", "C", "C", "B", "A", 
          "A", "C", "A", "A", "A", "C", "B", "C", "A", "B", 
          "C", "A", "B", "B", "B", "B", "B", "C", "A", "B"]
for i,j in zip(myAnswer, answer):
    if i == j:
        same += 1
print(same, "/", len(myAnswer))


119 / 200
